In [ ]:
!pip install PyArabic

In [1]:
import pyarabic.araby as araby
import pandas as pd
import numpy as np
from datasets import DatasetDict, Dataset
import re

/home/ahmadsameh8/anaconda3/envs/textCorrection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_csv("MergedDataset.csv")

dataset

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت على الكرسي ليظ...,سبحان الله الحكام العرب سيموت على الكرسي ليضهر...
1,النصر آت لا محال إن شاء الله . من يؤمن بالله و...,النصر آت لا محال انشاء الله من يؤمن بالله والي...
2,إلى كل شخص يعتقد أن بشار الأسد سوف يخرج من سور...,الى كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الأسد وعصابته - لحد الآن - غير مستوعبين أنه رح...,الأسد وعصابته لحد الآن غير مستوعبين إن و رح ين...
4,النظام الأسدي تجاوز حتى ما فعله معمر القذافي ف...,النظام الأسدي تجاوز حتى ما فعله معمر القذافي ف...
...,...,...
22933,"حوار عن "" الشرعية الدولية "" . في البرنامج "" ال...","حوار عن "" الشرعية الدولية "" في البرنامج "" الات..."
22934,استمر دكتور جمال قريب إلى نهاية محاضرته أن يصف...,استمر دكتور جمال قريب من نهاية محاضرته أن يصف ...
22935,أصبحت اللغة العربية في هذا العصر موضوع اهتمام ...,أصبحت اللغة العربية في ھذا العصر موضوع إھتمام ...
22936,الأقلية الفلسطينية في إسرائيل . حضرت محاضرة لل...,الأقل ة الفلسط ن ة ف إسرائ ل حضرت محاضرة للأست...


# preprocessing of the dataset

In [3]:
class preprocessing:
    import re

    def clean_text(text):
        """Cleans text by removing English characters, mentions, URLs, and hashtags (except those with trailing characters), while preserving whitespace."""
        cleaned_text = re.sub(r'[@A-Za-z]+|#[^\s]+|http\S+|@[^\s]+|\^[^ ]+', '', text)
        return cleaned_text


    def normalize(text):
        '''
        Argument:
            string of words
        return:
            string of words but standardize the words
        '''

        text = text.strip()
        text = re.sub("[إأٱآا]", "ا", text)
        text = re.sub("ى", "ي", text)
        text = re.sub("ؤ", "ء", text)
        text = re.sub("ئ", "ء", text)
        text = re.sub("ة", "ه", text)
        noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
        text = re.sub(noise, '', text)
        text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
        return araby.strip_tashkeel(text)
        
    
    def preprocess(sentence):
        '''
        Argument:
            string of words
        return:
            string of words after cleaning
        '''
        clean = preprocessing.clean_text(sentence)
        clean = preprocessing.normalize(clean)
        return clean


In [4]:
dataset["correct"] = dataset["correct"].apply(preprocessing.preprocess)
dataset["incorrect"] = dataset["incorrect"].apply(preprocessing.preprocess)
dataset

,correct,incorrect
0,سبحان الله ، الحكام العرب سيموت علي الكرسي ليظ...,سبحان الله الحكام العرب سيموت علي الكرسي ليضهر...
1,النصر ات لا محال ان شاء الله . من يءمن بالله و...,النصر ات لا محال انشاء الله من يءمن بالله والي...
2,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...,الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سور...
3,الاسد وعصابته - لحد الان - غير مستوعبين انه رح...,الاسد وعصابته لحد الان غير مستوعبين ان و رح ين...
4,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...,النظام الاسدي تجاوز حتي ما فعله معمر القذافي ف...
...,...,...
22933,"حوار عن "" الشرعيه الدوليه "" . في البرنامج "" ال...","حوار عن "" الشرعيه الدوليه "" في البرنامج "" الات..."
22934,استمر دكتور جمال قريب الي نهايه محاضرته ان يصف...,استمر دكتور جمال قريب من نهايه محاضرته ان يصف ...
22935,اصبحت اللغه العربيه في هذا العصر موضوع اهتمام ...,اصبحت اللغه العربيه في ھذا العصر موضوع اھتمام ...
22936,الاقليه الفلسطينيه في اسراءيل . حضرت محاضره لل...,الاقل ه الفلسط ن ه ف اسراء ل حضرت محاضره للاست...


# Converting the dataset to a hugging face Object


In [5]:
dataset_dict = {
    'correct': dataset["correct"].tolist(),
    'incorrect': dataset['incorrect'].tolist()
}
dataset_dict

{'correct': ['سبحان الله ، الحكام العرب سيموت علي الكرسي ليظهر انه عنيد وقوي . لو كان بشار يحب ارضه او شعبه ، لخرج من الحكم شفقه ورحمه ببلد ضاع . هنا زال قناع هذا الرءيس ، الذي خيب ظن شعبه والشعوب المسلمه ، كل مال السورين نفق في شراء سلاح ليقتل به ، شتان وحكام اوربا ، الذين يتركون الكرسي لمجرد فتنه بسيطه لحبهم لبلدهم .',
  'النصر ات لا محال ان شاء الله . من يءمن بالله واليوم الاخر ، والمطلع علي قصص الانبياء ، يري ان النصر من الله ، وعلامات القيامه اتيه في تسلسل عجيب . " ان تنصروا الله ينصركم " لا رجوع ، انما التغيير ات ليس من الرءساء ، انما من فوق سابع سماء .',
  'الي كل شخص يعتقد ان بشار الاسد سوف يخرج من سوريا بدون محكمه ، فهو وهم . ان بشار الاسد قتل الشعب سوري ، واقول لكل شخص سوري ، او غير سوري ، ان بشار وطغمته عثوا فسادا في سوريا 48 عاما ، يكفي والاسلام اي الاسد ، حكمت فافنيت بلادا وخيرا . اقول لكل شخص من الامن وجيش : اتق الله في الشعب سوري البطل',
  'الاسد وعصابته - لحد الان - غير مستوعبين انه رح ينقلعوا من فوق رقاب الشعب السوري . . . يعني لسع راكبه براسهم ان هذا الشعب قطيع غنم ، 

In [6]:

hf_dataset = Dataset.from_dict(dataset_dict)
hf_dataset


Dataset({
    features: ['correct', 'incorrect'],
    num_rows: 22938
})

In [7]:
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = dataset_size - train_size - val_size

In [8]:
train_dataset = hf_dataset.select(range(train_size))
validation_dataset = hf_dataset.select(range(train_size, train_size + val_size))
test_dataset = hf_dataset.select(range(train_size + val_size, len(hf_dataset)))

In [9]:
print("Training set size:", train_size)
print("Validation set size:", val_size)
print("Testing set size:", test_size)


Training set size: 18350
Validation set size: 2293
Testing set size: 2295


In [10]:
dataset = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test": test_dataset
})

for split_name, split_dataset in dataset.items():
    print(f"{split_name}: {split_dataset}")

# features = dataset["train"].features
# dataset = dataset.map(lambda example: {k: v for k, v in example.items() if k in features})

train: Dataset({
    features: ['correct', 'incorrect'],
    num_rows: 18350
})
validation: Dataset({
    features: ['correct', 'incorrect'],
    num_rows: 2293
})
test: Dataset({
    features: ['correct', 'incorrect'],
    num_rows: 2295
})


In [11]:

features = dataset["train"].features
dataset = dataset.map(lambda example: {k: v for k, v in example.items() if k in features})

Map: 100%|██████████| 2295/2295 [00:00<00:00, 56313.71 examples/s]


# Pushing the dataset to hugging face

In [17]:
!pip install huggingface_hub


In [21]:
# add hugging face token with access to write
# !huggingface-cli login

In [20]:
dataset.push_to_hub("QalbPreprocessedAndMerged")

Uploading the dataset shards: 100%|██████████| 1/1 [00:09<00:00,  9.86s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Ahmadsameh8/QalbPreprocessedAndMerged/commit/de07c50bf3f921cdefada0f7df993d4a7d40fbee', commit_message='Upload dataset', commit_description='', oid='de07c50bf3f921cdefada0f7df993d4a7d40fbee', pr_url=None, pr_revision=None, pr_num=None)